Intake-STAC + Landsat8
================

In this notebook, we'll take a look at some of the functionality in Intake-STAC by exploring an example STAC 1.0 catalog with a Landsat-8 L1 public data on AWS (https://registry.opendata.aws/landsat-8/)

In [ ]:
%matplotlib inline
import intake
print(intake.__version__)

In [ ]:
# for development
#%load_ext autoreload
#%autoreload 2

In [ ]:
# intake checks for registered drivers when imported
# You should see 'stac_catalog, stac_collection, stac_item, and stac_item_collection' if intake-stac is installed
list(intake.registry)

## Load root catalog

In [ ]:
# Open a catalog and list subcatalogs / collections
url = 'https://raw.githubusercontent.com/sat-utils/sat-stac/master/test/catalog/catalog.json'
cat = intake.open_stac_catalog(url)
cat.metadata.keys()
list(cat)

In [ ]:
# Drill down into subcatalogs
subcat = cat['stac-catalog-eo']
list(subcat)

In [ ]:
# Drill further down into subcatalogs
subcat1 = subcat['landsat-8-l1']
list(subcat1)

In [ ]:
subcat1.metadata

In [ ]:
# NOTE: would be good to have easy way for user to distinguish between catalogs and items
print(type(subcat1._stac_obj))
item = subcat1['LC08_L1TP_152038_20200611_20200611_01_RT']
print(type(item._stac_obj))

In [ ]:
item.metadata

In [ ]:
# Check assets
list(item)

In [ ]:
# Display thumbnail
from IPython.display import Image
Image(item['thumbnail'].urlpath)

In [ ]:
# Load single band into xarray data array
da = item.B4.to_dask()
da

## Stacking bands

In [ ]:
bands = ['nir','red']
stack = item.stack_bands(bands)
type(stack)

In [ ]:
# Bug? currently need to specify chunks:
da = stack(chunks=dict(band=1, x=2048, y=2048)).to_dask()

In [ ]:
da

In [ ]:
# Note that common names were mapped to standard names 'nir'->'B5', 'red'->'B4'
# An alternative organization is to store as a DataSet with common names:
da['band'] = bands
ds = da.to_dataset(dim='band')
ds

In [ ]:
# Now we can calculate band indices of subregions easily
NDVI = (ds['nir'] - ds['red']) / (ds['nir'] + ds['red'])

In [ ]:
NDVI.isel(y=slice(2000,3000), x=slice(1500,2000)).plot.imshow(cmap='BrBG', vmin=-1, vmax=1)